# 1. 패키지 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import folium
from folium.features import DivIcon
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json

# 2. 데이터 불러오기

* 해당 데이터의 경우 생활인구 현황을 확인하는 것으로써 별도의'count'컬럼을 주고 해당 값을 지도에 그렸습니다.
-> 본인 데이터에 맞게 수정 바랍니다.

In [3]:
# 생활인구
life_people = pd.read_csv('/content/drive/MyDrive/Miniproject_No_1/0.Code&Data/datas/생활인구/LOCAL_PEOPLE_DONG_202302.csv', index_col=False)

In [4]:
# 행정동 코드와 행정동명
dong_code = pd.read_csv('/content/drive/MyDrive/Miniproject_No_1/0.Code&Data/datas/생활인구/서울_행정동코드.csv',index_col=0)
dong_code.reset_index(inplace=True)

In [5]:
dong_code['행정동명'] = dong_code['행정동명'].str.replace('서울','서울특별시')
dong_code

,행정동코드,행정동명
0,11110530,서울특별시 종로구 사직동
1,11110540,서울특별시 종로구 삼청동
2,11110550,서울특별시 종로구 부암동
3,11110560,서울특별시 종로구 평창동
4,11110570,서울특별시 종로구 무악동
...,...,...
419,11740690,서울특별시 강동구 둔촌1동
420,11740700,서울특별시 강동구 둔촌2동
421,11740570,서울특별시 강동구 암사1동
422,11740610,서울특별시 강동구 천호2동


## 2-1. 데이터 정보 확인

In [6]:
life_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284928 entries, 0 to 284927
Data columns (total 32 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   기준일ID            284928 non-null  int64  
 1   시간대구분            284928 non-null  int64  
 2   행정동코드            284928 non-null  int64  
 3   총생활인구수           284928 non-null  float64
 4   남자0세부터9세생활인구수    284928 non-null  float64
 5   남자10세부터14세생활인구수  284928 non-null  float64
 6   남자15세부터19세생활인구수  284928 non-null  float64
 7   남자20세부터24세생활인구수  284928 non-null  float64
 8   남자25세부터29세생활인구수  284928 non-null  float64
 9   남자30세부터34세생활인구수  284928 non-null  float64
 10  남자35세부터39세생활인구수  284928 non-null  float64
 11  남자40세부터44세생활인구수  284928 non-null  float64
 12  남자45세부터49세생활인구수  284928 non-null  float64
 13  남자50세부터54세생활인구수  284928 non-null  float64
 14  남자55세부터59세생활인구수  284928 non-null  float64
 15  남자60세부터64세생활인구수  284928 non-null  float64
 16  남자65세부터69세생활인구수  284928 non-null  floa

In [7]:
dong_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정동코드   424 non-null    int64 
 1   행정동명    424 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.8+ KB


## 2-2. 데이터 merge(행정동코드를 통한 지역명 추가)

In [8]:
df_merge = pd.merge(life_people,dong_code, left_on='행정동코드', right_on='행정동코드', how="inner")
df_merge

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,행정동명
0,20230201,0,11650610,19374.1348,701.9605,369.2433,494.7090,496.0783,656.9863,746.4260,...,778.6073,828.8505,778.8992,838.3477,772.4319,796.6686,677.7395,585.4010,1402.0716,서울특별시 서초구 방배2동
1,20230201,1,11650610,18779.1080,660.1243,347.2364,488.7557,442.6008,658.9824,710.8183,...,761.5392,808.0663,788.7770,825.3218,756.4031,815.4799,663.8729,571.8624,1401.6874,서울특별시 서초구 방배2동
2,20230201,2,11650610,18457.8187,678.5871,356.9483,483.7914,436.1592,610.4463,674.6562,...,749.9360,777.2600,755.9941,822.1623,710.5197,775.0730,640.4445,579.0470,1387.5177,서울특별시 서초구 방배2동
3,20230201,3,11650610,18519.3378,700.8134,368.6396,457.7460,433.1332,602.0785,689.4438,...,744.3371,752.3800,762.0305,832.7452,736.2799,771.3669,628.3864,577.5756,1432.3291,서울특별시 서초구 방배2동
4,20230201,4,11650610,18879.2852,717.1447,377.2302,475.6419,458.8394,630.7275,675.6487,...,765.3629,779.4974,786.4446,834.2395,735.9232,782.9474,644.1242,608.7752,1421.7945,서울특별시 서초구 방배2동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284923,20230228,19,11380570,46161.4882,1642.1860,757.8083,1157.8517,1258.2765,1584.1436,1594.7055,...,2060.0602,2156.9106,1728.1041,1920.5337,1901.6056,2034.6589,1787.6976,1242.4637,3638.7179,서울특별시 은평구 대조동
284924,20230228,20,11380570,47104.8201,1618.3087,746.7893,1034.6478,1250.9622,1634.6608,1652.1632,...,2160.7746,2156.2672,1802.4624,1981.2997,1860.8871,2136.4417,1826.2038,1267.0547,3588.0007,서울특별시 은평구 대조동
284925,20230228,21,11380570,47441.1998,1625.9221,750.3033,990.3299,1330.5109,1683.2651,1726.3721,...,2127.8295,2306.2308,1769.0471,1884.9623,1858.7698,2127.6369,1816.1851,1236.0238,3601.0534,서울특별시 은평구 대조동
284926,20230228,22,11380570,49330.5070,1642.0670,757.7536,966.7454,1364.1889,1914.0025,1887.2880,...,2375.6919,2341.2542,1842.5258,1931.2810,1811.9668,2123.2040,1821.5920,1252.0599,3678.4273,서울특별시 은평구 대조동


## 2-2-1. 각 지역별(행정동별) 시간대별 1차 merge

In [9]:
df_merge_gu_hour = df_merge.groupby(['행정동명','시간대구분'], as_index=False).mean()
df_merge_gu_hour

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,서울특별시 강남구 개포1동,0,20230214.5,11680660.0,9394.493807,353.265029,224.039182,329.443114,192.231600,190.036343,...,187.036314,278.240768,408.381532,441.613996,515.009796,327.877511,319.553354,296.119582,306.774511,666.397614
1,서울특별시 강남구 개포1동,1,20230214.5,11680660.0,9035.547875,339.038539,215.016893,315.690564,187.569586,182.313807,...,181.496900,268.418132,398.161704,424.483971,493.580300,319.582496,303.963418,282.602739,291.018100,640.889954
2,서울특별시 강남구 개포1동,2,20230214.5,11680660.0,8947.434396,335.704232,212.902132,314.544825,189.563839,178.034993,...,179.974307,267.543971,396.790746,420.177107,488.216071,313.948689,302.305104,276.621107,288.570096,629.660393
3,서울특별시 강남구 개포1동,3,20230214.5,11680660.0,8884.018575,333.939132,211.782846,314.154625,191.719232,180.839829,...,177.921239,268.099961,392.851046,417.609046,481.867946,313.346700,297.076039,273.352332,285.992093,618.088096
4,서울특별시 강남구 개포1동,4,20230214.5,11680660.0,8982.141336,331.870864,210.471161,321.254889,193.651961,183.740146,...,178.202675,265.790379,394.077657,420.636411,487.982543,315.984854,312.624107,276.718914,282.588857,612.026989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,서울특별시 중랑구 중화2동,19,20230214.5,11260610.0,27873.553732,1056.809443,475.710325,511.478629,717.117589,855.521554,...,943.075011,866.105993,1125.732600,905.451925,1025.803379,1047.698946,1330.581936,1283.980611,954.197464,2273.199854
10172,서울특별시 중랑구 중화2동,20,20230214.5,11260610.0,28258.895854,1053.409096,474.179636,508.581068,727.438021,887.241779,...,980.639811,914.655271,1158.220868,913.497529,1039.033504,1073.526811,1350.667404,1283.218682,957.152443,2260.738146
10173,서울특별시 중랑구 중화2동,21,20230214.5,11260610.0,28525.366664,1058.210711,476.341025,528.897746,734.225511,914.168732,...,1021.952843,945.373096,1183.171779,916.768686,1043.831818,1077.763214,1356.801743,1272.947468,953.428396,2254.224271
10174,서울특별시 중랑구 중화2동,22,20230214.5,11260610.0,28697.051161,1019.214257,458.787300,538.377954,753.155636,944.285193,...,1069.325339,971.350082,1201.520689,917.244879,1067.903071,1088.802825,1375.917614,1267.390729,957.536475,2242.318836


In [10]:
df_merge_gu_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10176 entries, 0 to 10175
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   행정동명             10176 non-null  object 
 1   시간대구분            10176 non-null  int64  
 2   기준일ID            10176 non-null  float64
 3   행정동코드            10176 non-null  float64
 4   총생활인구수           10176 non-null  float64
 5   남자0세부터9세생활인구수    10176 non-null  float64
 6   남자10세부터14세생활인구수  10176 non-null  float64
 7   남자15세부터19세생활인구수  10176 non-null  float64
 8   남자20세부터24세생활인구수  10176 non-null  float64
 9   남자25세부터29세생활인구수  10176 non-null  float64
 10  남자30세부터34세생활인구수  10176 non-null  float64
 11  남자35세부터39세생활인구수  10176 non-null  float64
 12  남자40세부터44세생활인구수  10176 non-null  float64
 13  남자45세부터49세생활인구수  10176 non-null  float64
 14  남자50세부터54세생활인구수  10176 non-null  float64
 15  남자55세부터59세생활인구수  10176 non-null  float64
 16  남자60세부터64세생활인구수  10176 non-null  float64
 17  남자65세부터69세생활

In [11]:
df_merge_gu_hour['시간대구분'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

## 2-2-2. 야간 생활인구 데이터 만들기
주간 - 06시\~17시<br>
야간 - 18시\~05시

In [12]:
# 시간대 별 분리
df_merge_gu_hour_yagan = df_merge_gu_hour[(5 <= df_merge_gu_hour['시간대구분']) | (df_merge_gu_hour['시간대구분'] >= 18)]
df_merge_gu_hour_yagan 

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
5,서울특별시 강남구 개포1동,5,20230214.5,11680660.0,9196.228396,322.231111,204.357682,309.230379,194.586011,189.867279,...,178.893804,263.443064,394.429350,426.133382,474.240121,312.543614,312.759421,275.533889,279.191196,601.909796
6,서울특별시 강남구 개포1동,6,20230214.5,11680660.0,10283.433507,333.841146,211.720711,326.782646,230.979971,263.215221,...,195.921579,272.208689,414.508554,453.148932,503.307414,333.745032,344.250379,283.706271,288.955439,618.159457
7,서울특별시 강남구 개포1동,7,20230214.5,11680660.0,9991.868646,332.710768,211.003804,314.693921,217.581764,253.832389,...,188.362468,261.006446,394.458193,442.244471,529.542939,327.631339,343.885061,293.004464,293.598454,617.455625
8,서울특별시 강남구 개포1동,8,20230214.5,11680660.0,9279.478082,345.730239,219.260664,296.683725,195.963682,231.532754,...,177.069082,234.212925,368.604511,400.973764,467.782425,306.300354,341.009604,280.830768,293.410075,583.065711
9,서울특별시 강남구 개포1동,9,20230214.5,11680660.0,8989.031507,345.327471,219.005236,271.777343,188.180364,226.090632,...,181.158321,221.671393,361.287246,398.820707,455.170350,313.910496,346.761000,296.323807,291.607586,576.539907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,서울특별시 중랑구 중화2동,19,20230214.5,11260610.0,27873.553732,1056.809443,475.710325,511.478629,717.117589,855.521554,...,943.075011,866.105993,1125.732600,905.451925,1025.803379,1047.698946,1330.581936,1283.980611,954.197464,2273.199854
10172,서울특별시 중랑구 중화2동,20,20230214.5,11260610.0,28258.895854,1053.409096,474.179636,508.581068,727.438021,887.241779,...,980.639811,914.655271,1158.220868,913.497529,1039.033504,1073.526811,1350.667404,1283.218682,957.152443,2260.738146
10173,서울특별시 중랑구 중화2동,21,20230214.5,11260610.0,28525.366664,1058.210711,476.341025,528.897746,734.225511,914.168732,...,1021.952843,945.373096,1183.171779,916.768686,1043.831818,1077.763214,1356.801743,1272.947468,953.428396,2254.224271
10174,서울특별시 중랑구 중화2동,22,20230214.5,11260610.0,28697.051161,1019.214257,458.787300,538.377954,753.155636,944.285193,...,1069.325339,971.350082,1201.520689,917.244879,1067.903071,1088.802825,1375.917614,1267.390729,957.536475,2242.318836


In [13]:
# 야간 평균 인구수 계산
df_merge_gu_hour_yagan_mean = df_merge_gu_hour_yagan.groupby(['행정동명'], as_index=False).mean()
df_merge_gu_hour_yagan_mean

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,서울특별시 강남구 개포1동,14.0,20230214.5,11680660.0,8537.767005,316.574470,200.770211,250.146368,166.315352,191.083521,...,172.597733,231.656994,367.702769,403.570492,457.036348,318.330626,327.971636,277.297305,281.048992,573.245020
1,서울특별시 강남구 개포2동,14.0,20230214.5,11680670.0,25416.964012,1085.462151,913.166006,1035.814391,650.064314,641.756596,...,531.076798,715.086963,1026.584825,1169.276123,1419.596296,1005.085465,1079.117621,848.400372,741.684589,1932.380591
2,서울특별시 강남구 개포4동,14.0,20230214.5,11680690.0,15568.645481,490.129275,310.794591,447.628213,399.647785,545.702137,...,528.188116,550.291632,724.821889,634.385998,734.315111,685.194039,638.660259,493.081327,474.628289,857.918458
3,서울특별시 강남구 논현1동,14.0,20230214.5,11680521.0,48066.920278,599.142584,244.188204,570.668785,1492.548293,2938.698045,...,4121.500096,3460.071124,3031.550762,2137.082705,1760.264184,1238.048850,1188.377661,859.123074,740.882819,1683.051021
4,서울특별시 강남구 논현2동,14.0,20230214.5,11680531.0,49118.194099,924.245379,344.512880,620.683345,1078.245132,2568.891370,...,3844.360444,3476.639389,3125.330822,2303.452958,1917.830129,1300.196948,1189.512384,885.293705,683.043641,1813.327897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,서울특별시 중랑구 상봉2동,14.0,20230214.5,11260590.0,26213.687472,612.171708,258.995321,455.161646,860.352645,1273.925098,...,1525.379552,1405.760674,1319.358960,926.002368,968.115712,1003.965770,1170.836033,965.175206,693.222268,1635.206168
420,서울특별시 중랑구 신내1동,14.0,20230214.5,11260680.0,34797.332179,2176.210523,895.125198,869.717670,865.095000,973.747835,...,950.092088,1124.827889,1365.648169,1349.562102,1299.899601,1344.420600,1484.408679,1348.438814,882.069186,2600.717051
421,서울특별시 중랑구 신내2동,14.0,20230214.5,11260690.0,16951.538492,709.257762,413.159053,527.963780,419.337594,421.776648,...,494.247151,538.986617,672.670488,635.065869,708.422534,677.981316,830.865766,717.939801,508.983483,1442.290205
422,서울특별시 중랑구 중화1동,14.0,20230214.5,11260600.0,13748.451694,695.111120,276.230163,345.365905,351.435860,415.898070,...,452.525605,517.491975,530.087520,447.176132,552.124758,558.809353,654.689013,572.390826,499.000129,1165.823184


In [14]:
# 성동구만
df_merge_gu_hour_yagan_seondong_mean = df_merge_gu_hour_yagan_mean[df_merge_gu_hour_yagan_mean['행정동명'].str.contains('성동구')]
df_merge_gu_hour_yagan_seondong_mean

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
244,서울특별시 성동구 금호1가동,14.0,20230214.5,11200590.0,14522.936120,606.138651,334.548697,365.409408,298.097831,332.160182,...,379.346111,537.215238,901.442030,778.910135,696.363437,522.846812,577.291816,549.274782,456.830775,1055.330274
245,서울특별시 성동구 금호2.3가동,14.0,20230214.5,11200615.0,19835.398639,1085.639815,351.587787,455.816419,440.917423,532.156654,...,532.054584,761.765100,1128.050169,934.298187,836.273349,759.098817,847.903697,765.386905,561.280043,1711.533776
246,서울특별시 성동구 금호4가동,14.0,20230214.5,11200620.0,11648.570725,527.089586,172.554879,212.050998,264.154770,274.715923,...,313.865763,437.768303,751.659665,551.933052,506.226742,434.007635,448.126987,443.646095,337.151086,908.005211
247,서울특별시 성동구 마장동,14.0,20230214.5,11200540.0,21127.521227,568.579097,309.471230,456.370996,805.544782,900.722791,...,868.836176,777.350702,853.325928,690.843268,808.877856,841.171166,890.002080,738.186242,544.682921,1542.725716
248,서울특별시 성동구 사근동,14.0,20230214.5,11200550.0,23706.133664,401.542301,183.055907,631.507329,2346.399279,2040.600590,...,1633.102226,840.128490,829.674663,628.399229,779.386220,617.464565,644.554007,527.846062,386.675651,965.789396
249,서울특별시 성동구 성수1가1동,14.0,20230214.5,11200650.0,20500.327895,476.435381,229.312335,315.757894,649.133565,992.350267,...,1234.227201,1174.220473,1171.950582,740.783449,761.045899,695.538614,764.938984,619.131507,418.208461,776.310899
250,서울특별시 성동구 성수1가2동,14.0,20230214.5,11200660.0,21830.654950,443.337640,199.967365,327.426378,685.068656,1155.303679,...,1570.370088,1384.531634,1186.613612,869.913342,805.467045,634.503227,627.903652,514.335303,361.523665,801.332994
251,서울특별시 성동구 성수2가1동,14.0,20230214.5,11200670.0,20688.522791,527.344775,235.218053,262.183159,580.127770,1014.935078,...,1041.181042,919.421860,1024.473215,711.732921,674.542642,756.518414,842.487373,671.407717,493.622881,935.347822
252,서울특별시 성동구 성수2가3동,14.0,20230214.5,11200690.0,39634.509231,606.988574,201.777998,487.196739,1470.630515,2568.419363,...,3193.315311,2291.609762,2066.569803,1456.667585,1388.850883,1263.027891,1120.661129,816.031081,544.006399,1197.255794
253,서울특별시 성동구 송정동,14.0,20230214.5,11200720.0,9719.411751,174.431245,88.206623,156.316548,380.207247,566.648060,...,571.981425,418.797103,467.533042,324.422934,338.269830,313.008674,329.108763,270.326623,194.456160,549.104937


# 3. 지도 그리기

## 3-1. geojson 파일 불러오기(단계 구분도는 선을 그려주기 위함임.)

In [15]:
r = requests.get('https://raw.githubusercontent.com/vuski/admdongkor/master/ver20230101/HangJeongDong_ver20230101.geojson')
c = r.content
seoul_dong_geo = json.loads(c)

In [16]:
type(seoul_dong_geo)

dict

In [17]:
seoul_dong_geo

Output hidden; open in https://colab.research.google.com to view.

In [18]:
seoul_dong_geo_result = {'type': 'FeatureCollection',
                         'name': 'HangJeongDong_ver20230101',
                         'crs': {'type': 'name',
                         'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
                         'features': [feature for feature in seoul_dong_geo['features'] if '서울특별시' in feature['properties']['adm_nm']]}

In [19]:
seoul_dong_geo_result

{'type': 'FeatureCollection',
 'name': 'HangJeongDong_ver20230101',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'adm_nm': '서울특별시 종로구 사직동',
    'adm_cd': '1101053',
    'adm_cd2': '1111053000',
    'sgg': '11110',
    'sido': '11',
    'sidonm': '서울특별시',
    'temp': '종로구 사직동',
    'sggnm': '종로구',
    'adm_cd8': '11010530'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[126.97688884274817, 37.575650779448786],
       [126.9770344988775, 37.56919453005455],
       [126.97597472821249, 37.56933629942577],
       [126.97537470991254, 37.56931556702156],
       [126.97433193562325, 37.56926180051753],
       [126.96904837001854, 37.56819441770833],
       [126.96854493603384, 37.56842767961275],
       [126.9666499598212, 37.56949165520658],
       [126.96628175024485, 37.569700734798694],
       [126.9660973270804, 37.56985650972371],
       [126.96572852922577, 3

In [20]:
# 성동구만
seoul_dong_geo_seongdong_result = {'type': 'FeatureCollection',
                         'name': 'HangJeongDong_ver20230101',
                         'crs': {'type': 'name',
                         'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
                         'features': [feature for feature in seoul_dong_geo_result['features'] if '성동구' in feature['properties']['adm_nm']]}

In [21]:
seoul_dong_geo_seongdong_result

{'type': 'FeatureCollection',
 'name': 'HangJeongDong_ver20230101',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 49,
    'adm_nm': '서울특별시 성동구 왕십리2동',
    'adm_cd': '1104052',
    'adm_cd2': '1120052000',
    'sgg': '11200',
    'sido': '11',
    'sidonm': '서울특별시',
    'temp': '성동구 왕십리2동',
    'sggnm': '성동구',
    'adm_cd8': '11040520'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.03351058909426, 37.562517856120664],
       [127.03106512462858, 37.560333620339904],
       [127.03058050927122, 37.560309204050505],
       [127.03036692165554, 37.56040299634883],
       [127.03024780478002, 37.560439703139274],
       [127.02819595211548, 37.56027040521961],
       [127.02808576665176, 37.56023729093031],
       [127.02707040142865, 37.55977128643338],
       [127.02677837997646, 37.55961005467636],
       [127.0267478281066, 37.559591003875354],
       [127.0267129

## 3-2. folium 패키지에 내장된 leaflet.js 지도 불러오기

In [22]:
# 서울시 중심부의 위도, 경도 입니다.
center = [37.5634, 127.0364]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center, zoom_start=13)

In [23]:
df_merge_gu_hour_yagan_mean['행정동명']

0      서울특별시 강남구 개포1동
1      서울특별시 강남구 개포2동
2      서울특별시 강남구 개포4동
3      서울특별시 강남구 논현1동
4      서울특별시 강남구 논현2동
            ...      
419    서울특별시 중랑구 상봉2동
420    서울특별시 중랑구 신내1동
421    서울특별시 중랑구 신내2동
422    서울특별시 중랑구 중화1동
423    서울특별시 중랑구 중화2동
Name: 행정동명, Length: 424, dtype: object

In [ ]:
df_merge_gu_hour_yagan_mean[df_merge_gu_hour_yagan_mean['총생활인구수'] == df_merge_gu_hour_yagan_mean['총생활인구수']]

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,서울특별시 강남구 개포1동,14.0,20230214.5,11680660.0,8537.767005,316.574470,200.770211,250.146368,166.315352,191.083521,...,172.597733,231.656994,367.702769,403.570492,457.036348,318.330626,327.971636,277.297305,281.048992,573.245020
1,서울특별시 강남구 개포2동,14.0,20230214.5,11680670.0,25416.964012,1085.462151,913.166006,1035.814391,650.064314,641.756596,...,531.076798,715.086963,1026.584825,1169.276123,1419.596296,1005.085465,1079.117621,848.400372,741.684589,1932.380591
2,서울특별시 강남구 개포4동,14.0,20230214.5,11680690.0,15568.645481,490.129275,310.794591,447.628213,399.647785,545.702137,...,528.188116,550.291632,724.821889,634.385998,734.315111,685.194039,638.660259,493.081327,474.628289,857.918458
3,서울특별시 강남구 논현1동,14.0,20230214.5,11680521.0,48066.920278,599.142584,244.188204,570.668785,1492.548293,2938.698045,...,4121.500096,3460.071124,3031.550762,2137.082705,1760.264184,1238.048850,1188.377661,859.123074,740.882819,1683.051021
4,서울특별시 강남구 논현2동,14.0,20230214.5,11680531.0,49118.194099,924.245379,344.512880,620.683345,1078.245132,2568.891370,...,3844.360444,3476.639389,3125.330822,2303.452958,1917.830129,1300.196948,1189.512384,885.293705,683.043641,1813.327897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,서울특별시 중랑구 상봉2동,14.0,20230214.5,11260590.0,26213.687472,612.171708,258.995321,455.161646,860.352645,1273.925098,...,1525.379552,1405.760674,1319.358960,926.002368,968.115712,1003.965770,1170.836033,965.175206,693.222268,1635.206168
420,서울특별시 중랑구 신내1동,14.0,20230214.5,11260680.0,34797.332179,2176.210523,895.125198,869.717670,865.095000,973.747835,...,950.092088,1124.827889,1365.648169,1349.562102,1299.899601,1344.420600,1484.408679,1348.438814,882.069186,2600.717051
421,서울특별시 중랑구 신내2동,14.0,20230214.5,11260690.0,16951.538492,709.257762,413.159053,527.963780,419.337594,421.776648,...,494.247151,538.986617,672.670488,635.065869,708.422534,677.981316,830.865766,717.939801,508.983483,1442.290205
422,서울특별시 중랑구 중화1동,14.0,20230214.5,11260600.0,13748.451694,695.111120,276.230163,345.365905,351.435860,415.898070,...,452.525605,517.491975,530.087520,447.176132,552.124758,558.809353,654.689013,572.390826,499.000129,1165.823184


In [ ]:
df_merge_gu_hour_yagan_mean['총생활인구수'].nlargest(30)

16     124593.863161
341    110602.490509
203     85931.487192
385     77561.175839
239     71722.702067
124     69688.023838
375     58339.376416
217     58333.532554
17      57933.499896
241     57698.276335
356     57355.740979
25      56179.571665
342     55431.554597
6       55362.984254
300     54929.088136
53      54427.890188
238     52581.457984
201     52555.226336
123     51062.491874
22      50182.291847
4       49118.194099
3       48066.920278
230     47722.041515
289     47651.917856
396     47644.160512
10      47466.841498
15      47287.912290
117     46760.807578
308     46755.094570
21      46679.518361
Name: 총생활인구수, dtype: float64

In [ ]:
#df_merge_gu_hour_jugan_mean['총생활인구수'].nlargest(30)
df_merge_gu_hour_yagan_mean.sort_values(by=['총생활인구수'],ascending=False).head(30)

,행정동명,시간대구분,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
16,서울특별시 강남구 역삼1동,14.0,20230214.5,11680640.0,124593.863161,1109.924042,483.353233,2329.768297,4210.950326,8451.193210,...,10527.071992,8957.417856,7540.988070,5156.480594,4123.423559,2826.221777,2458.594657,1994.137225,1491.482862,2953.908303
341,서울특별시 영등포구 여의동,14.0,20230214.5,11560540.0,110602.490509,1865.182519,900.554052,1296.657494,2162.544136,5634.788604,...,6691.100857,6396.018155,6886.802993,5006.596047,4492.163713,2847.818854,2741.586239,2037.639214,1566.369663,3619.092437
203,서울특별시 마포구 서교동,14.0,20230214.5,11440660.0,85931.487192,1056.291943,500.519789,2362.457273,5428.541973,5888.205916,...,8008.126616,5259.189163,4135.403550,2855.456733,2775.427871,2104.755062,1896.622119,1435.216403,1016.536651,2499.151110
385,서울특별시 종로구 종로1.2.3.4가동,14.0,20230214.5,11110615.0,77561.175839,1086.803010,342.417710,791.022283,2410.599521,3788.765384,...,4428.335788,3857.539782,3636.930347,3150.476542,3150.286660,2523.431425,2445.957380,1927.494978,1332.580693,2933.407386
239,서울특별시 서초구 서초3동,14.0,20230214.5,11650530.0,71722.702067,1572.695241,997.080604,1889.451676,1781.397918,2913.670093,...,3894.152337,3793.413748,3818.424140,3037.735736,3345.970159,2578.476191,2216.558235,1732.841951,1282.777498,2832.981278
124,서울특별시 금천구 가산동,14.0,20230214.5,11545510.0,69688.023838,659.111767,226.390752,553.003953,2128.985041,5375.950804,...,5277.039702,4115.017179,3466.489405,2545.779357,2502.005124,1900.774823,1876.482931,1282.147820,802.537413,1392.426614
375,서울특별시 은평구 진관동,14.0,20230214.5,11380690.0,58339.376416,3368.098264,2126.136727,2345.407821,1427.781428,1259.302100,...,1802.643200,1778.815831,2421.765611,2575.597640,3086.265798,2262.276544,2219.033652,1937.754377,1504.415752,4223.210701
217,서울특별시 서대문구 신촌동,14.0,20230214.5,11410585.0,58333.532554,814.619417,256.060922,1339.384812,4409.879880,4051.971401,...,5553.380120,3145.526275,2298.874123,1650.233236,1835.398108,1636.624373,1625.520745,1157.497767,854.775435,2057.233737
17,서울특별시 강남구 역삼2동,14.0,20230214.5,11680650.0,57933.499896,2076.246223,1414.404109,1882.080258,1410.418806,2447.825002,...,3162.685815,2716.017831,2867.738756,2914.495928,3252.101636,1905.483468,1387.870274,1051.308857,918.009841,2012.894291
241,서울특별시 서초구 양재1동,14.0,20230214.5,11650651.0,57698.276335,3387.048742,1203.492362,1065.069620,1135.547043,2238.631709,...,2370.545406,2496.131301,2639.914725,2541.494177,2364.807401,1840.966655,1792.631008,1623.457822,1270.785400,3047.713055


## 3-3. folium 패키지에 내장된 Choropleth 통해 단계구분도 그리기

In [23]:
# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=seoul_dong_geo_seongdong_result, # geo_json 파일
    data=df_merge_gu_hour_yagan_seondong_mean, # 해당 데이터 넣기
    columns=('행정동명', '총생활인구수'), # 해당하는 값
    key_on='feature.properties.adm_nm', # geo_json = key값(지역명 있는 json 키값 확인할 것)
    fill_color='BuGn', # 채우기 색
    legend_name='야간 생활인구 수', # 범례 이름 넣기기
).add_to(m) # 위에 불러온 지도에 덧칠한다는 마인드

# 맵 m을 출력합니다.
m

# 4. 지도에 텍스트 넣기

In [ ]:
seoul_gu.info()

NameError: ignored

## 4.1 지역명 + 위,경도 딕셔너리 만들기 -> 들어갈 텍스트의 위치 잡기 위함임.

In [ ]:
seoul_panmae5 = {}
for index in seoul_gu.index:
    seoul_panmae5[seoul_gu.loc[index, '시군구명'] ] = (seoul_gu.loc[index, "위도"], seoul_gu.loc[index, "경도"])

In [ ]:
seoul_panmae5

## 4.2 지도에 텍스트 넣기

In [ ]:
# (필요시) 데이터 약간 수정(넣을 텍스트를 위해)
seoul_panmae4["idx"] = seoul_panmae4["도로명주소"].str.split().str[1]
seoul_panmae4.set_index("idx", inplace=True)

In [ ]:
for key, value in seoul_panmae5.items():
    folium.map.Marker(
        # 위경도 위치
        [value[0], value[1]],  

        # DivIcon 을 사용
        # html 태그를 이용해서 text를 올릴 수 있음
        icon=DivIcon(
            # icon px 사이즈
            icon_size=(0, 0),
            # icon 좌 상단 위치 설정
            icon_anchor=(0, 0),

            # html 형식으로 text 추가
            # div 태그 안에 style 형식 추가
            html='<div\
                    style="\
                        font-size: 1.5rem;\
                        color: black;\
                        background-color:rgba(255, 255, 255, 0.2);\
                        width:120px;\
                        text-align:left;\
                        margin:0px;\
                    "><b>'
            + key + ': ' + str(seoul_panmae4.loc[key, 'count'])
        )).add_to(m)

In [ ]:
m